In [1]:
from datasets import Dataset, DatasetDict, load_dataset
import ast
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

/workspace/llm-fine-tuning/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from unsloth import FastLanguageModel

In [2]:

def get_data(csv_path, valid_set_ratio = 0.15):
    dataset = load_dataset('csv', data_files="arcelik_llm_training_set.csv")

    # Split the dataset into training and evaluation sets
    train_test_split = dataset['train'].train_test_split(test_size=0.15)
    dataset = DatasetDict({
        'train': train_test_split['train'],
        'eval': train_test_split['test']
    })
    return dataset

In [15]:

csv_path = "./arcelik_llm_training_set.csv"
base_model_id = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
max_seq_length = 130
load_in_4bit =True

In [16]:
def get_model_and_tokenizer(model_id: str, max_length, load_in_4bit,):
    model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id, # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )
    return model, tokenizer

In [17]:
model, tokenizer = get_model_and_tokenizer(base_model_id,
                                           max_seq_length,
                                           load_in_4bit)

config.json: 100%|██████████| 1.07k/1.07k [00:00<00:00, 1.68MB/s]


==((====))==  Unsloth: Fast Mistral patching release 2024.2
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.679 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = False.
 "-____-"     Apache 2 free license: http://github.com/unslothai/unsloth


/workspace/llm-fine-tuning/venv/lib/python3.10/site-packages/transformers/quantizers/auto.py:147: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be prevail.
  warnings.warn(warning_msg)
model.safetensors:  64%|██████▍   | 2.65G/4.13G [02:56<01:02, 23.7MB/s]

In [5]:

dataset = get_data(csv_path)

tokenizer = get_tokenizer(base_model_id)

PROMPT_TEMPLATE = "[INST] Extract span of text from the customer review associated with the topic - {}. Customer Review : '{}'[/INST] {} "

Generating train split: 26854 examples [00:00, 222769.54 examples/s]
tokenizer_config.json: 100%|██████████| 1.47k/1.47k [00:00<00:00, 5.82MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 18.4MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 32.2MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 357kB/s]


In [9]:
max_length = 120

def format_row(row):
    indices = ast.literal_eval(row['indices'])
    span = row['sentence'][indices[0]:indices[1]]
    try:
        assert span == row['span']
    except Exception:
        print(row['sentence'])
        print(span)
        print(row['span'])
        print(indices)
        print("next")
    # formatted_string = f"### Question : Extract span of text from the customer review associated with the topic - {row['topic']}. Customer Review - '{row['sentence']}'\n ### Answer: {span} "
    formatted_string = PROMPT_TEMPLATE.format(row['topic'], row['sentence'], span)
    result = tokenizer(formatted_string,truncation=True,
        max_length=max_length,
        padding="max_length",)
    new_record = {}
    result["labels"] = result["input_ids"].copy()
    # result = {}
    # result["labels"] = formatted_string
    del row['sentence']
    del row['sentence_id']
    del row['topic']
    del row['sentiment']
    del row['span_in_sentence']
    del row['span']
    del row['indices']
    return result

In [10]:
tokenized_train_dataset = dataset['train'].map(format_row)
tokenized_val_dataset = dataset['eval'].map(format_row)

Map:   0%|          | 0/22825 [00:00<?, ? examples/s]

Map:  43%|████▎     | 9914/22825 [00:03<00:03, 3360.18 examples/s]

Pros: Space saving for a small kitchen like mine Cons: None
None
None
(55, 59)
next


Map:  95%|█████████▌| 21794/22825 [00:08<00:00, 2453.07 examples/s]

Pros: Its easy to access and space saving Cons: None
None
None
(48, 52)
next


Map: 100%|██████████| 4029/4029 [00:01<00:00, 2364.31 examples/s]


In [11]:
tokenized_train_dataset[0]

{'input_ids': [2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  733,
  16289,
  28793,
  1529,
  2107,
  12363,
  302,
  2245,
  477,
  272,
  6346,
  4058,
  5363,
  395,
  272,
  9067,
  387,
  8382,
  28723,
  16648,
  8349,
  714,
  464,
  28737,
  403,
  6416,
  11572,
  304,
  315,
  2613,
  298,
  737,
  456,
  18401,
  28705,
  562,
  378,
  28735,
  776,
  459,
  354,
  528,
  1815,
  28792,
  28748,
  16289,
  28793,
  459,
  354,
  528,
  28705,
  2],
 'attention_mask': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,